In [40]:
from jax import hessian, jit, vmap, grad, jacobian, value_and_grad
import jax.numpy as np

In [46]:
def dummy_model(params, xy):
    result1 = np.sum(params * xy[1]*xy[0]**4) + np.sum(params * xy[0]*xy[1]**5)
    result2 = np.square(xy[1]**5) + np.square(xy[0]**10) * params
    return np.array([result1, result2])
                
psi = lambda params, xy: dummy_model(params, xy)[0]

process_hessian = lambda hess: np.array([[-hess[1, 1], hess[1, 0]], [hess[1, 0], -hess[0, 0]]])

hess_psi = hessian(psi, argnums=1)
d_psi_dxy = grad(psi, argnums=1)

advection_term = lambda params, xy: process_hessian(hess_psi(params, xy)) @ d_psi_dxy(params, xy)

grad_hess_psi = jacobian(hess_psi, argnums=1)
u_x = lambda params, xy: hess_psi(params, xy)[0,1] #psi_yx
u_y = lambda params, xy: hess_psi(params, xy)[1,1] #psi_yy


u_xx = lambda params, xy: grad(u_x, argnums=1)(params, xy)[0] #psi_yxx
u_yy = lambda params, xy: grad(u_y, argnums=1)(params, xy)[1] #psi_yyy

#we can be cheecky with by using value_and_grad

v_y = lambda params, xy: -u_x(params, xy) #-psi_xy=-psi_yx
v_x = lambda params, xy: -hess_psi(params, xy)[0,0] #-psi_xx

process_gradient = lambda gradient: np.array((gradient[0], gradient[1][0]))
#-psi_xx and -psi_xxy = -psi_yxx = u_xx
v_xANDu_xx = lambda params, xy: process_gradient(value_and_grad(v_x, argnums=1)(params, xy))





In [53]:
# Test the advection_term function
params = np.array(1., dtype=np.float32)
xy = np.array([1., 1.], dtype=np.float32)

v_xANDu_xx_result = v_xANDu_xx(params, xy)
result_model = dummy_model(params, xy)

#result_psi = psi(params, xy)
result_hess = hess_psi(params, xy)

result_adv = advection_term(params, xy)
grad_hess_psi_result = grad_hess_psi(params, xy)

#print(v_xANDu_xx_result)
print(grad_hess_psi_result[0,0,0])

24.0


In [2]:
import numpy as np
n = 1
for i in range(n-1):
    print("jippi")